In [2]:
#michael is sus

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as stat

In [3]:
company_tweet = pd.read_csv('Data/Tweets/Company_Tweet.csv')
tweet = pd.read_csv('Data/Tweets/Tweet.csv')
company = pd.read_csv('Data/Tweets/Company.csv')
company_value = pd.read_csv('Data/CompanyStocks/CompanyValues.csv')

In [4]:
tweet = tweet.dropna()
#merge company tweets and tweets on id
tweet_df = pd.merge(company_tweet , tweet , on="tweet_id", how= "inner")

In [5]:
#create variable 'total_engagement'
tweet_df["total_engangement"] = tweet_df["comment_num"] + tweet_df["retweet_num"] + tweet_df["like_num"]
#remove tweets below certain engagement
engagement_threshold = 40
tweet_df = tweet_df[tweet_df["total_engangement"] > engagement_threshold]
tweet_df.head()
print('tweet count: ', tweet_df.size)

tweet count:  485118


In [6]:
#convert original time on tweet_df to date time
tweet_df['post_date'] = pd.to_datetime(tweet_df['post_date'], unit='s')
tweet_df['day_date'] = pd.to_datetime(tweet_df['post_date'].apply(lambda date: date.date()))

In [7]:
#order chronilogically
tweet_df = tweet_df.sort_values(by="day_date")

In [8]:
#convert time value on company stock data to date time
stock_df = company_value
stock_df['day_date']  = pd.to_datetime(stock_df['day_date'])
stock_df.sample(5)


,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
8680,GOOGL,2010-06-09,239.4488,2724728,246.1167,246.9552,238.4284
5892,AMZN,2017-07-22,1025.6700,2726844,1021.2800,1026.1000,1011.0000
4156,AMZN,2016-02-26,555.2300,4870280,560.1200,562.5000,553.1700
16487,TSLA,2012-05-11,32.2500,1221005,32.4900,33.4400,32.1600
6445,GOOGL,2019-04-26,1277.4200,1357955,1273.3800,1278.9100,1265.0000


In [9]:
#drop un-needed features
tweet_df = tweet_df.drop(['tweet_id', 'post_date', 'comment_num', 'retweet_num', 'like_num'], axis=1)

In [10]:
#remove any stock data that is outside of the tweet data
start_day = min(tweet_df['day_date'])
print(f"Start day: {start_day}")

end_day = max(tweet_df['day_date'])
print(f"End day: {end_day}")

Start day: 2015-01-01 00:00:00
End day: 2019-12-31 00:00:00


In [11]:
stock_df = stock_df[(stock_df['day_date'] >= start_day) & (stock_df['day_date'] <= end_day)]

In [12]:
#sort stock data chronilogically
stock_df = stock_df.sort_values(by="day_date")
stock_df.head()

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
2517,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.21
10810,GOOG,2015-01-01,526.40,1367110,531.25,532.60,525.80
8687,GOOGL,2015-01-01,530.66,1235889,537.74,538.40,530.20
16960,TSLA,2015-01-01,222.41,2392947,223.09,225.68,222.25
5602,AMZN,2015-01-01,310.35,2048676,311.55,312.98,310.01


In [13]:
tweet_df.head()

,ticker_symbol,writer,body,total_engangement,day_date
40,AAPL,CNBC,This is Wall Street's top pick in 2015. Hint: ...,64,2015-01-01
47,AAPL,WSJ,"See how tech companies like Apple, Twitter and...",347,2015-01-01
239,AAPL,greatspoke,Top Stock Analyst Releases Report on Top Stock...,433,2015-01-01
240,AAPL,greatspoke,Analyst Report on Top Stock Market Options for...,433,2015-01-01
241,AMZN,greatspoke,Analyst Report on Top Stock Market Options for...,433,2015-01-01


In [35]:
APPL_df = stock_df.loc[stock_df['ticker_symbol'] == 'AAPL']

GOOG_df = stock_df.loc[stock_df['ticker_symbol'] == 'GOOG']
GOOGL_df = stock_df.loc[stock_df['ticker_symbol'] == 'GOOGL']


TSLA_df = stock_df.loc[stock_df['ticker_symbol'] == 'TSLA']
AMZN_df = stock_df.loc[stock_df['ticker_symbol'] == 'AMZN']
MSFT_df = stock_df.loc[stock_df['ticker_symbol'] == 'MSFT']

In [15]:
#there are two types of google stocks, one has voting power in the company while the other doesnt
GOOGL_df.head()

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
8687,GOOGL,2015-01-01,530.66,1235889,537.74,538.4000,530.20
7530,GOOGL,2015-01-02,529.55,1325271,532.60,535.8000,527.88
8688,GOOGL,2015-01-03,529.55,1325271,532.60,535.8000,527.88
8689,GOOGL,2015-01-04,529.55,1325271,532.60,535.8000,527.88
7529,GOOGL,2015-01-05,519.46,2054120,527.15,527.9899,517.75


In [16]:
APPL_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'AAPL']

In [17]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [18]:
import nltk                             
# from nltk.corpus import twitter_samples                            
# from nltk.corpus import stopwords 
# from nltk.stem import PorterStemmer
# from nltk.tokenize import TweetTokenizer  

import random  
import re                     
import string


In [19]:
def remove_special_character(tweet):
#     print(tweet)
    # remove the old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # remove hashtags. only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)

    # remove single numeric terms in the tweet. 
    tweet = re.sub(r'[0-9]', '', tweet)
    
    return tweet

In [20]:
tweet_df["body"] = tweet_df["body"].apply(lambda tweet: remove_special_character(tweet))
tweet_df["body"] = tweet_df["body"].str.lower()
tweet_df.sample(5)

# ?? why is the body text so fucked up ??

,ticker_symbol,writer,body,total_engangement,day_date
1381733,AAPL,charliebilello,"revenue growth, year-over-year (%)... $fb: +...",92,2018-08-01
4037039,TSLA,vincent13031925,“tesla victorious after judge dismisses $tsla ...,302,2019-03-25
3830988,TSLA,markbspiegel,summary of the $gs reinstatement on $tsla. the...,101,2018-09-04
3044526,AMZN,SJosephBurns,"top s&p stocks, last yrs:netflix $nflx: +,%m...",126,2019-09-17
1383385,AAPL,zachmil_es,my original iphone on ios .. even knows how to...,669,2018-08-02


In [21]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [22]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vanderSentimentAnalyzer = SentimentIntensityAnalyzer()

In [23]:
def getVanderScore(tweet):    
    vs = vanderSentimentAnalyzer.polarity_scores(tweet)
    score = vs['compound']
    return score

In [24]:
tweet_df['vander_score'] = tweet_df['body'].apply(lambda tweet: getVanderScore(tweet))


In [25]:
#create a weighted scored based off engagement and sentiment analysis score
tweet_df['summarative_score'] = tweet_df['total_engangement'] * tweet_df['vander_score'] * 0.01
tweet_df = tweet_df.drop(['total_engangement', 'vander_score'], axis=1)


In [26]:
#sum or avg the summarative_scores for one date
APPL_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'AAPL']

GOOG_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'GOOG']
GOOGL_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'GOOGL']


TSLA_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'TSLA']
AMZN_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'AMZN']
MSFT_tweet = tweet_df.loc[tweet_df['ticker_symbol'] == 'MSFT']

In [27]:
GOOG_tweet.head()

,ticker_symbol,writer,body,day_date,summarative_score
1850635,GOOG,WSJ,google now controls or occupies a strip of man...,2015-01-04,0.000000
4040,GOOG,DanielEran,"despite lawsuit, $aapl ios is far more storag...",2015-01-05,0.410522
1854377,GOOG,WSJ,google is exploring how to use frequencies to ...,2015-01-20,0.000000
1854055,GOOG,WSJ,google is close to investing $ billion in spac...,2015-01-20,1.547000
1854431,GOOG,The_Real_Fly,"obama is long $ebay, $goog and $tsla",2015-01-21,0.000000


In [28]:
APPL_tweet = APPL_tweet.groupby('day_date')['summarative_score'].sum()
GOOG_tweet = GOOG_tweet.groupby('day_date')['summarative_score'].sum()
GOOGL_tweet = GOOGL_tweet.groupby('day_date')['summarative_score'].sum()
TSLA_tweet = TSLA_tweet.groupby('day_date')['summarative_score'].sum()
AMZN_tweet = AMZN_tweet.groupby('day_date')['summarative_score'].sum()
MSFT_tweet = MSFT_tweet.groupby('day_date')['summarative_score'].sum()


In [29]:
APPL_df = APPL_df.merge(APPL_tweet, how = 'outer', on = 'day_date')
GOOG_df = APPL_df.merge(GOOG_tweet, how = 'outer', on = 'day_date')
GOOGL_df = APPL_df.merge(GOOGL_tweet, how = 'outer', on = 'day_date')
TSLA_df = APPL_df.merge(TSLA_tweet, how = 'outer', on = 'day_date')
AMZN_df = APPL_df.merge(AMZN_tweet, how = 'outer', on = 'day_date')
MSFT_df = APPL_df.merge(MSFT_tweet, how = 'outer', on = 'day_date')

APPL_df.head()

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value,summarative_score
0,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.21,3.911989
1,AAPL,2015-01-02,109.33,53143770,111.39,111.44,107.35,2.044350
2,AAPL,2015-01-03,109.33,53143770,111.39,111.44,107.35,NaN
3,AAPL,2015-01-04,109.33,53143770,111.39,111.44,107.35,1.740136
4,AAPL,2015-01-05,106.25,64210880,108.29,108.65,105.41,0.866415


In [30]:
APPL_df.replace(to_replace = np.nan, value = 0)
GOOG_df.replace(to_replace = np.nan, value = 0)
GOOGL_df.replace(to_replace = np.nan, value = 0)
AMZN_df.replace(to_replace = np.nan, value = 0)
TSLA_df.replace(to_replace = np.nan, value = 0)
MSFT_df.replace(to_replace = np.nan, value = 0)


,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value,summarative_score_x,summarative_score_y
0,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.21,3.911989,0.000000
1,AAPL,2015-01-02,109.33,53143770,111.39,111.44,107.35,2.044350,0.000000
2,AAPL,2015-01-03,109.33,53143770,111.39,111.44,107.35,0.000000,0.000000
3,AAPL,2015-01-04,109.33,53143770,111.39,111.44,107.35,1.740136,0.000000
4,AAPL,2015-01-05,106.25,64210880,108.29,108.65,105.41,0.866415,0.410522
...,...,...,...,...,...,...,...,...,...
1821,AAPL,2019-12-27,289.80,36592940,291.12,293.97,288.12,0.115701,1.539251
1822,AAPL,2019-12-28,289.80,36592940,291.12,293.97,288.12,0.411740,0.116788
1823,AAPL,2019-12-29,289.80,36592940,291.12,293.97,288.12,2.102808,2.693193
1824,AAPL,2019-12-30,291.52,36059610,289.46,292.69,285.22,7.807207,0.000000


# # LSTM MODEL: 

In [31]:
#pip install tensorflow==2.9.1

In [32]:
#here we will do the LSTM model

#/-\|/-\|
import math
import tensorflow 
from tensorflow import keras as ks
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Dropout, LSTM
from keras.layers import Dense, LSTM

from keras.optimizers import Adam

plt.style.use('fivethirtyeight')


In [37]:
APPL_df.head(10)

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
2517,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.210
1360,AAPL,2015-01-02,109.33,53143770,111.39,111.44,107.350
2518,AAPL,2015-01-03,109.33,53143770,111.39,111.44,107.350
2519,AAPL,2015-01-04,109.33,53143770,111.39,111.44,107.350
1359,AAPL,2015-01-05,106.25,64210880,108.29,108.65,105.410
1358,AAPL,2015-01-06,106.26,65682250,106.54,107.43,104.630
1357,AAPL,2015-01-07,107.75,39983350,107.20,108.20,106.695
1356,AAPL,2015-01-08,111.89,59168760,109.23,112.15,108.700
1355,AAPL,2015-01-09,112.01,53645730,112.67,113.25,110.210
2520,AAPL,2015-01-10,112.01,53645730,112.67,113.25,110.210


In [60]:
forecast_out= 3
x1 = APPL_df.iloc[: , 3:]
APPL_df['PrevNClose'] = APPL_df[['close_value']].shift(forecast_out)
#print(df[:10])

X = pd.concat([x1,x3] , axis = 1)
X.pop("close_value")
#X = X.drop(labels = None, axis = 0, index = 'close_value')
X= X[forecast_out:]
print(X)

y = APPL_df[['close_value']]
y = y[forecast_out:]
print(y)

        volume  open_value  high_value  low_value  PrevNClose
2519  53143770      111.39      111.44    107.350      110.38
1359  64210880      108.29      108.65    105.410      109.33
1358  65682250      106.54      107.43    104.630      109.33
1357  39983350      107.20      108.20    106.695      109.33
1356  59168760      109.23      112.15    108.700      106.25
...        ...         ...         ...        ...         ...
105   36592940      291.12      293.97    288.120      284.27
3083  36592940      291.12      293.97    288.120      284.27
3084  36592940      291.12      293.97    288.120      289.91
104   36059610      289.46      292.69    285.220      289.80
103   25247630      289.93      293.68    289.520      289.80

[1823 rows x 5 columns]
      close_value
2519       109.33
1359       106.25
1358       106.26
1357       107.75
1356       111.89
...           ...
105        289.80
3083       289.80
3084       289.80
104        291.52
103        293.65

[1823 rows x 1

/var/folders/c8/kyj_xhqd1hz487fy48brhp1m0000gp/T/ipykernel_30589/1603753570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  APPL_df['PrevNClose'] = APPL_df[['close_value']].shift(forecast_out)


In [62]:
#Scale Features
scaler = MinMaxScaler(feature_range = (0,1))
X = scaler.fit_transform(X)
X.shape


[[0.27758168 0.10635442 0.09772615 0.08937766 0.10041589]
 [0.35110731 0.09094073 0.08393475 0.07968008 0.09515458]
 [0.36088252 0.08223946 0.0779041  0.07578105 0.09515458]
 ...
 [0.16762433 1.         1.         0.99300175 1.        ]
 [0.16408109 0.99174622 0.99367276 0.97850537 0.99944881]
 [0.09225046 0.99408313 0.99856649 1.         0.99944881]]


In [53]:
#Split Train and Test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


x_train.shape


(1458, 5, 1)

In [54]:

model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = ( x_train.shape[1],1 )))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))


#model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
#model.add(Dense(1))
#model.compile(optimizer='adam', loss='mse')



2022-06-27 18:30:59.491707: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [56]:
#Compile model
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#Train model
#xt_small = x_train[:1000]
#yt_small = y_train[:1000]
#model.fit(x_train, y_train, batch_size =1 , epochs = 3)
model.fit(x_train, y_train, batch_size =1 , epochs = 10)


Epoch 1/10
1458/1458 [==============================] - 11s 5ms/step - loss: 10.2040
Epoch 2/10
1458/1458 [==============================] - 8s 5ms/step - loss: 10.5420
Epoch 3/10
1458/1458 [==============================] - 8s 5ms/step - loss: 8.9941
Epoch 4/10
1458/1458 [==============================] - 9s 6ms/step - loss: 8.9095
Epoch 5/10
1458/1458 [==============================] - 7s 5ms/step - loss: 7.4864
Epoch 6/10
1458/1458 [==============================] - 8s 5ms/step - loss: 7.8364
Epoch 7/10
1458/1458 [==============================] - 8s 5ms/step - loss: 6.3332
Epoch 8/10
1458/1458 [==============================] - 8s 5ms/step - loss: 8.2675
Epoch 9/10
1458/1458 [==============================] - 8s 5ms/step - loss: 7.4327
Epoch 10/10
1458/1458 [==============================] - 8s 5ms/step - loss: 6.0695


In [69]:
#Test Dataset
x_test= np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
#print(x_test.shape)



predictions = model.predict(x_test)
print(predictions.shape)

#print(predictions)

pred = np.zeros(predictions.shape)
pred = np.array(pred)
pred = np.concatenate( [pred, pred, pred, pred, predictions] , axis = 1)

#predictions = scaler.inverse_transform(predictions)
predictions = scaler.inverse_transform(pred)
predictions = predictions[:, 4]

predictions = np.reshape(predictions, (y_test.shape[0], y_test.shape[1]))
rmse = np.sqrt( np.mean(( predictions - y_test) ** 2))
print(rmse)

12/12 [==============================] - 0s 6ms/step
(365, 1)
close_value    32077.394839
dtype: float64


/Users/jaydenlu/opt/anaconda3/envs/tensor-flow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
